In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import pylab
import os 

In [2]:
defen = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', sheetname=1, encoding = 'utf8').append(pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheetname=1, encoding='utf8')).append(pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', sheetname=1, encoding='utf8'))

In [137]:
def D_maker(data):
    #rez = pd.DataFrame()
    #k=1
    rez=[]
    for index in data.index:
        for column in data.columns:
            i_tmp = index.split(':')
            c_tmp = column.split(':')
            value=[]
            if i_tmp[2]!=u'p-value' and i_tmp[0]!=c_tmp[0]:
                value.append([data.get_value(index=index, col=column),data.get_value(index=i_tmp[0]+':'+i_tmp[1]+':p-value', col=column)])
                if value[0][0]>0.5 and value[0][1]<0.1 and value[0][0]!=1:
                    #rez.append(pd.DataFrame(np.array(value[0][0]), columns=['Cor'], index=[k]))
                    #rez.append(pd.DataFrame(defen[defen['Code']==i_tmp[0]]['Indicator Name'].tolist()[0], columns=['Def1'], index=[k]))
                    #rez.append(pd.DataFrame(defen[defen['Code']==c_tmp[0]]['Indicator Name'].tolist()[0], columns=['Def2'], index=[k]))
                    #k+=1
                    rez.append([defen[defen['Code']==i_tmp[0]]['Indicator Name'].tolist()[0], defen[defen['Code']==c_tmp[0]]['Indicator Name'].tolist()[0], value[0][0]])
                    #Degbug                    
                    #return rez 
    return pd.DataFrame().append(rez)

def Sumup(d_name):
    files = os.listdir(d_name)
    for name in files:
        data = pd.read_excel(d_name+'/'+name ,encoding = 'utf8')
        n_regino = name.split('.')[1].split('_')[2]
    return D_maker(data)

In [138]:
tmp = Sumup('all_results')
len(tmp)

4532

In [134]:
tmp.head()

,0,1,2
0,"Inflation, consumer prices (annual %)",Number of infant deaths,0.910993
1,"Inflation, consumer prices (annual %)",Number of under-five deaths,0.910781
2,"Inflation, consumer prices (annual %)",Number of neonatal deaths,0.907715
3,"Inflation, consumer prices (annual %)","Mortality rate, under-5 (per 1,000 live births)",0.665343
4,"Inflation, consumer prices (annual %)","Mortality rate, neonatal (per 1,000 live births)",0.675374


In [87]:
#for d in ['all_results','data1_results', 'data2_results','data3_results']:
#    Sumup(d)

In [88]:
'all.Corr_in_Africa.xlsx'.split('.')[1].split('_')[2]

'Africa'

In [89]:
test = pd.read_excel('all_results'+'/'+'all.Corr_in_Africa.xlsx' ,encoding = 'utf8')

In [90]:
test.index[1].split(':')[0]

'FP.CPI.TOTL.ZG'

In [91]:
test.columns[0].split(':')

['FP.CPI.TOTL.ZG', 'Africa']

In [92]:
test.index[0]

'FP.CPI.TOTL.ZG:Africa:cor-value'

In [93]:
defen[defen['Code']=='FP.CPI.TOTL.ZG']['Indicator Name'].tolist()[0]

'Inflation, consumer prices (annual %)'

In [94]:
defen.head()

,Code,Indicator Name,Long definition,Source
0,SH.HIV.ARTC.FE.ZS,Antiretroviral therapy coverage (% of adult fe...,The percentage of adult females living with HI...,UNAIDS estimates.
1,SH.HIV.ARTC.MA.ZS,Antiretroviral therapy coverage (% of adult ma...,The percentage of adult males living with HIV ...,UNAIDS estimates.
2,SP.POP.DPND,Age dependency ratio (% of working-age populat...,Age dependency ratio is the ratio of dependent...,World Bank staff estimates using the World Ban...
3,SP.POP.AG00.FE.IN,"Age population, age 0, female, interpolated","Age population, female refers to female popula...",The United Nations Population Division’s World...
4,SP.POP.AG00.MA.IN,"Age population, age 0, male, interpolated","Age population, male refers to male population...",The United Nations Population Division’s World...
